In [1]:
import json
import jsonlines


In [2]:
with open("/root/code/ChatGLM_PEFT/main/data/sample_dev_pred.json", 'r', encoding='utf-8') as f:
    obj1 = json.load(f)
    print(type(obj1['answer']))

<class 'dict'>


In [21]:
# 打开JSONL文件
# with open("/root/code/ChatGLM_PEFT/output.txt", "r", encoding="utf8") as f:

# 读取 .jsonl 文件并将其转换为 Python 对象列表
from json import JSONDecodeError


def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            # 读取两行
            answer_line = json.loads(file.readline().strip())
            sp_line = json.loads(file.readline().strip())
            
            # 如果两行都为空，表示文件已经读完，退出循环
            if not answer_line and not sp_line:
                break
            try:
                answer = json.loads(str(answer_line).strip())  # 逐行读取并转换为 Python 对象
                # print(answer)
                # print("answer_type:"+str(type(answer)))
            except JSONDecodeError:
                try:
                    clean_line = sp_line.strip().strip('"').strip()
                    answer = clean_line
                except Exception:
                    answer = "<|>解析错误<|>"
                    print("答案解析错误") 
            except TypeError:
                answer = "<|>解析错误<|>"
                print("答案解析错误")

            try:
                sp = json.loads(sp_line)
                print(sp)
                print("sp_type"+str(type(sp)))
                # data.append(obj1) 
            except JSONDecodeError :
                print(sp_line)
                # 绝大多数格式错误都是少了前面的 [
                sp_line = "[" + str(sp_line)
                try:
                    sp = json.loads(sp_line)
                    
                    print("JSONDecode_sp_type" + str(type(sp)))
                    # data.append(obj1)   
                except JSONDecodeError:
                    #    其他类型错误，直接舍弃
                    sp=[]
            except TypeError:
                sp=[]
            record = {
                "answer":answer,
                
            }
    return data

# 使用示例
file_path = '/root/code/ChatGLM_PEFT/output.txt'  # 将其替换为你的 .jsonl 文件路径
data = read_jsonl(file_path)
print("结束")
# # 打印读取的内容
# for obj in data:
#     print(obj + str(type(obj)))


yes
answer_type:<class 'str'>
[['Scott Derrickson', 'A0'], ['Ed Wood', 'I0']]
sp_type<class 'list'>
United States ambassador
answer_type:<class 'str'>
["Shirley Temple", "K0"], ["Kiss and Tell (1945 film)", "E0"]]
JSONDecode_sp_type<class 'list'>
Animorphs
answer_type:<class 'str'>
["Victoria Hanley", "A0"], ["Animorphs", "J0"], ["Animorphs", "K0"]]
JSONDecode_sp_type<class 'list'>
no
answer_type:<class 'str'>
["Laleli Mosque", "L0"], ["Esma Sultan Mansion", "M0"]]
JSONDecode_sp_type<class 'list'>
Greenwich Village
answer_type:<class 'str'>
["Adriana Trigiani", "A0"], ["Big Stone Gap (film)", "I0"]]
JSONDecode_sp_type<class 'list'>
YG Entertainment
answer_type:<class 'str'>
[['2014 S/S', 'L0'], ['Winner (band)', 'A0']]
sp_type<class 'list'>
Eenasul Fateh
answer_type:<class 'str'>
["Eenasul Fateh", "P0"], ["Management consulting", "Q0"]]
JSONDecode_sp_type<class 'list'>
4,000
answer_type:<class 'str'>
["Lewiston Maineiacs", "A0"], ["Androscoggin Bank Colisée", "I0"]]
JSONDecode_sp_type<